In [25]:
# import library
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy import stats
import math
import statistics
import seaborn as sns
from IPython.display import display
import sklearn

%matplotlib inline




# 特徵選取會用到的函數
from sklearn.feature_selection import VarianceThreshold
from sklearn import preprocessing

from sklearn.datasets import make_friedman1
from sklearn.feature_selection import RFE
from sklearn.svm import SVC

#讀入資料
df_train = pd.read_csv('HWData/Titanic_train.csv')
print(df_train.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB
None


In [26]:
#取出資料後，把遺失值刪除
complete_data=df_train.dropna()

In [27]:
'''
Q1：目標變數為 Survived，以 Pclass, Age, SibSp, Parch, Fare,Sex, Embarked 為特徵，
試著用今天教授的包裝法，搭配課程所教的 SVC，試著排出其餘特徵的重要性。
'''
#先刪除沒有要列入包裝法選擇的特徵
complete_data=complete_data.drop(['Name','Ticket','PassengerId','Cabin'], axis=1)
display(complete_data.head(5))

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
1,1,1,female,38.0,1,0,71.2833,C
3,1,1,female,35.0,1,0,53.1000,S
6,0,1,male,54.0,0,0,51.8625,S
10,1,3,female,4.0,1,1,16.7000,S
11,1,1,female,58.0,0,0,26.5500,S


In [28]:
#1-定義x,y
x=complete_data[['Pclass', 'Age', 'SibSp', 'Parch', 'Fare','Sex', 'Embarked']]
y=complete_data['Survived']
display(x.head(5))
display(y.head(5))

,Pclass,Age,SibSp,Parch,Fare,Sex,Embarked
1,1,38.0,1,0,71.2833,female,C
3,1,35.0,1,0,53.1000,female,S
6,1,54.0,0,0,51.8625,male,S
10,3,4.0,1,1,16.7000,female,S
11,1,58.0,0,0,26.5500,female,S


1     1
3     1
6     0
10    1
11    1
Name: Survived, dtype: int64

In [29]:
#將離散數值轉化為連續數值:['Sex', 'Embarked']
sex_mapping = {
           'male': 1,
           'female': 0}
complete_data['Sex1'] = complete_data['Sex'].map(sex_mapping)

Embarked_mapping = {
           'C': 0,
           'S': 1}
complete_data['Embarked1'] = complete_data['Embarked'].map(Embarked_mapping)

complete_data['Age'] = round(complete_data['Age'],2)

complete_data['Fare'] = round(complete_data['Fare'],2)

x=complete_data[['Pclass', 'Age', 'SibSp', 'Parch', 'Fare','Sex1', 'Embarked1']]
display(x.head(5))

,Pclass,Age,SibSp,Parch,Fare,Sex1,Embarked1
1,1,38.0,1,0,71.28,0,0.0
3,1,35.0,1,0,53.10,0,1.0
6,1,54.0,0,0,51.86,1,1.0
10,3,4.0,1,1,16.70,0,1.0
11,1,58.0,0,0,26.55,0,1.0


In [33]:
print(x.info())
np.any(np.isnan(x))
x['Embarked1'] = x['Embarked1'].fillna(0)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 183 entries, 1 to 889
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Pclass     183 non-null    int64  
 1   Age        183 non-null    float64
 2   SibSp      183 non-null    int64  
 3   Parch      183 non-null    int64  
 4   Fare       183 non-null    float64
 5   Sex1       183 non-null    int64  
 6   Embarked1  183 non-null    float64
dtypes: float64(3), int64(4)
memory usage: 11.4 KB
None


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [34]:
#包裝法
estimator = SVC(kernel="linear")
selector = RFE(estimator, n_features_to_select=2, step=1)
selector = selector.fit(x, y)
## True= selected feature
print(selector.support_)
#Selected (i.e., estimated best) features are assigned rank 1.
ranking=selector.ranking_
print(ranking)
rfe_feature = x.loc[:,selector.support_].columns.tolist()
print(rfe_feature)

#找出最重要的2個特徵值為Sex, Embarked

[False False False False False  True  True]
[3 5 2 4 6 1 1]
['Sex1', 'Embarked1']
